In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathstuff import get_path
path=get_path()


In [3]:
import torchaudio
import torch
from pathlib import Path
from models.dataset import N2NDataset
from models.noise2noiseunet import train,Noise2NoiseUNet
from torch.utils.data import DataLoader
import noise_addition_utils
import numpy as np
np.random.seed(999)
torch.manual_seed(999)

c:\Users\Revathy\Desktop\WL_POC\Noise2Noise\noise_addition_utils.py:15: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [4]:
torchaudio.set_audio_backend("soundfile")
input_train_dir="../Datasets/WhiteNoise_Train_Input"
output_train_dir="../Datasets/WhiteNoise_Train_Output"

C:\Users\Revathy\AppData\Local\Temp\ipykernel_16396\2473502035.py:1: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [5]:
sample_rate = 48000
n_fft = (sample_rate * 64) // 1000 
hop_length = (sample_rate * 16) // 1000 


In [6]:

train_input =sorted( list(Path(input_train_dir).iterdir()))
train_target = sorted(list(Path(output_train_dir).iterdir()))

In [7]:
train_on_gpu=torch.cuda.is_available()      
DEVICE = torch.device('cuda' if train_on_gpu else 'cpu')

In [8]:
train_data=N2NDataset(train_input, train_target, n_fft, hop_length)
train_dataloader = DataLoader(train_data, batch_size=2, shuffle=True)

In [9]:
n2n = Noise2NoiseUNet(n_fft, hop_length).to(DEVICE)
optimizer = torch.optim.Adam(n2n.parameters())

In [16]:
loss_per_epoch = train(n2n, train_dataloader, optimizer,n_fft, hop_length)

  0%|          | 0/10 [00:00<?, ?it/s]

torch.Size([2, 32, 1531, 215, 2])
torch.Size([2, 32, 1531, 209, 2])
torch.Size([2, 64, 763, 103, 2])
torch.Size([2, 64, 379, 99, 2])
torch.Size([2, 64, 188, 49, 2])
torch.Size([2, 64, 92, 47, 2])
torch.Size([2, 64, 44, 23, 2])
torch.Size([2, 64, 20, 21, 2])
torch.Size([2, 64, 8, 10, 2])
torch.Size([2, 128, 2, 8, 2])
i 0
Decoder :  torch.Size([2, 64, 8, 10, 2])
i 1
Decoder :  torch.Size([2, 64, 20, 21, 2])
i 2
Decoder :  torch.Size([2, 64, 44, 23, 2])
i 3
Decoder :  torch.Size([2, 64, 92, 48, 2])


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 48 but got size 47 for tensor number 1 in the list.

In [ ]:
test_input = sorted(list(Path("Samples/Sample_Test_Input").rglob('*.wav')))
test_output = sorted(list(Path("Samples/Sample_Test_Target").rglob('*.wav')))

test_dataset = N2NDataset(test_input, test_output, n_fft, hop_length)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
n2n.eval()
test_loader_single_unshuffled_iter = iter(test_loader)

x_n, x_c = next(test_loader_single_unshuffled_iter)
for _ in range(4):
    x_n, x_c = next(test_loader_single_unshuffled_iter)

x_est = n2n(x_n, is_istft=True)

In [ ]:
x_est_np = x_est[0].view(-1).detach().cpu().numpy()
x_c=torch.view_as_complex(x_c)
x_n=torch.view_as_complex(x_n)
x_c_np = torch.istft(torch.squeeze(x_c[0], 1), n_fft=n_fft, hop_length=hop_length, normalized=True).view(-1).detach().cpu().numpy()
x_n_np = torch.istft(torch.squeeze(x_n[0], 1), n_fft=n_fft, hop_length=hop_length, normalized=True).view(-1).detach().cpu().numpy()